In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, pipeline
from torch.utils.data import DataLoader
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForQuestionAnswering, AutoTokenizer
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForQuestionAnswering
import torch.optim as optim
import torch.nn as nn
from torch import optim
import pandas as pd

In [ ]:
table_path = "/content/input_table.csv"
data_df = pd.read_csv(table_path)

In [ ]:
knowledge_base = "\n".join(data_df.astype(str).apply(lambda row: " | ".join(row), axis=1))

In [ ]:
data_df["context"] = data_df.astype(str).apply(lambda row: " ".join(row), axis=1)

In [ ]:
data_df.head()

,Invoice ID,City,Gender,Product line,Unit price,Quantity,Total,Date,Payment,gross income,Rating,context
0,750-67-8428,Yangon,Female,Health and beauty,74.69,7,548.9715,1/5/2019,Ewallet,26.1415,9.1,750-67-8428 Yangon Female Health and beauty 74...
1,226-31-3081,Naypyitaw,Female,Electronic accessories,15.28,5,80.2200,3/8/2019,Cash,3.8200,9.6,226-31-3081 Naypyitaw Female Electronic access...
2,631-41-3108,Yangon,Male,Home and lifestyle,46.33,7,340.5255,3/3/2019,Credit card,16.2155,7.4,631-41-3108 Yangon Male Home and lifestyle 46....
3,123-19-1176,Yangon,Male,Health and beauty,58.22,8,489.0480,1/27/2019,Ewallet,23.2880,8.4,123-19-1176 Yangon Male Health and beauty 58.2...
4,373-73-7910,Yangon,Male,Sports and travel,86.31,7,634.3785,2/8/2019,Ewallet,30.2085,5.3,373-73-7910 Yangon Male Sports and travel 86.3...


In [ ]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_df, tokenizer):
        self.data = data_df.reset_index(drop=True)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      row = self.data.iloc[idx]
      question = "What can you tell me about this entry?"
      context = f"Invoice ID: {row['Invoice ID']}, City: {row['City']}, Gender: {row['Gender']}, " \
                f"Product Line: {row['Product line']}, Unit Price: {row['Unit price']}, " \
                f"Quantity: {row['Quantity']}, Total: {row['Total']}, Date: {row['Date']}, " \
                f"Payment: {row['Payment']}, Gross Income: {row['gross income']}, Rating: {row['Rating']}."

      encoding = self.tokenizer(question, context, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

      return {
          "input_ids": encoding["input_ids"].squeeze(0),
          "attention_mask": encoding["attention_mask"].squeeze(0),
          "token_type_ids": encoding.get("token_type_ids", None).squeeze(0) if encoding.get("token_type_ids", None) is not None else None
      }


In [ ]:
train_dataset = CustomDataset( data_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch.get("token_type_ids", None)
        optimizer.zero_grad()
        if token_type_ids is not None:
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        else:
            outputs = model(input_ids, attention_mask=attention_mask)
        start_logits, end_logits = outputs.start_logits, outputs.end_logits
        start_positions = torch.zeros(start_logits.shape[0], dtype=torch.long)
        end_positions = (torch.sum(attention_mask, dim=1) - 1)
        loss = criterion(start_logits, start_positions) + criterion(end_logits, end_positions)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

KeyboardInterrupt: 

In [ ]:
def predict_answer(question, context):
    model.eval()
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)
    tokens = inputs["input_ids"].squeeze()
    answer = tokenizer.decode(tokens[start_idx:end_idx + 1])
    return answer

In [ ]:
question = "hat is the average unit price for transactions involving Cash payment method and a rating of 9.1?"
context = "The dataset contains transaction records with various details such as Payment method, Rating, and Unit price. For transactions where the Payment method is 'Cash' and the Rating is 9.1, the average Unit price is 75.32."

print("Answer:", predict_answer(question, context))

In [ ]:
inputs = tokenizer("What is the average unit price for transactions where the payment method is Cash and the rating is 9.1?", context, return_tensors="pt")
outputs = model(**inputs)

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx])

print("Final Model Prediction:", answer)